<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=0bc95307f722f29ad271b3053081ca9660a0d4186435b326b39ffa9d5a0d4421
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-20 11:46:36
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.41 C
-------------------
Today PnL: 33.91 K (0.24%)
Current PnL: -20.90 L (-13.92%)
CY Booked + Current PnL: -9.00 L (-5.99%)
-------------------
Total profit:  2.05 L
Total loss:  -22.94 L
-------------------
Total Booked + Current PnL: 17.98 L (14.61%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.46%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.06 L (62.64%)
Deployed:  1.23 C
Current:  1.41 C
CAGR/XIRR %: 7.84%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-17.38,67.0,X-LC,3.99,220707.0,16207.0,7085.0,2.12,7.93,3.21,11.39,10.0,2.29,1.58,17.59,X40N,ATH,PHARMA
56,RELIANCE,1533.00,-9.16,73.0,X-LC,2.88,227760.0,17394.0,11388.0,3.05,8.27,5.00,13.68,37.0,1.53,1.63,26.16,XY25,NTT,REFINERIES
83,VOLTAS,1530.00,2.14,61.0,X-MC,1.52,214350.0,22608.0,15155.0,0.17,11.79,7.07,19.69,99.0,1.49,1.54,19.39,XY25,NTT,AC
76,TTKPRESTIG,770.00,94.88,46.0,M-SC,1.72,83925.0,-16852.0,16944.0,-0.24,-16.72,20.19,0.09,245.0,-0.99,0.60,10.26,OX40N,NTT,DURABLES
33,ICICIGI,2260.25,-13.14,66.0,X-MC,2.68,149391.0,13258.0,17867.0,0.89,9.74,11.96,22.86,91.0,0.74,1.07,23.58,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANKINDIA,190.00,-22.98,65.0,H-MC,12.57,199033.0,19225.0,93187.0,5.12,10.69,46.82,62.52,88.0,0.21,1.43,48.42,XR,NTT,BANKS
10,BANDHANBNK,400.00,-10.58,54.0,H-SC,2.91,223512.0,-55047.0,312492.0,3.62,-19.76,139.81,92.42,151.0,-0.18,1.60,29.17,XY24,NTT,BANKS
78,UNIONBANK,163.00,-11.63,59.0,M-LC,8.79,161033.0,20193.0,24300.0,3.61,14.34,15.09,31.59,66.0,0.83,1.15,44.62,XY24,NTT,BANKS
6,ASIANTILES,137.00,7016.67,66.0,L-SC,6.46,78073.0,-15737.0,92353.0,3.24,-16.78,118.29,81.67,269.0,-0.17,0.56,51.19,XR,NTT,CERAMICS
45,KPIGREEN,731.05,4.41,64.0,H-SC,5.85,124274.0,-1023.0,59950.0,3.07,-0.82,48.24,47.03,141.0,-0.02,0.89,55.60,MH,ATH,POWER


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,TANLA,1963.11,-26.08,38.0,H-SC,18.75,198135.0,-72626.0,402571.0,-4.01,-26.82,203.18,121.86,133.0,-0.18,1.42,55.00,AR,ATH,IT
31,HINDZINC,730.22,29.80,52.0,M-LC,10.09,211171.0,6095.0,106472.0,-2.96,2.97,50.42,54.89,52.0,0.06,1.51,27.78,X5K,ATH,METALS
84,WHIRLPOOL,2270.00,-40.03,63.0,M-SC,4.16,101332.0,9834.0,68916.0,-2.41,10.75,68.01,86.07,223.0,0.14,0.73,47.15,XR,NTT,DURABLES
69,SYMPHONY,1306.00,-45.89,36.0,M-SC,18.98,115522.0,-55619.0,55566.0,-1.78,-32.50,48.10,-0.03,196.0,-1.00,0.83,0.71,OX40N,NTT,DURABLES
67,STARHEALTH,761.00,21.02,66.0,H-SC,9.78,269552.0,-11571.0,144426.0,-1.68,-4.12,53.58,47.26,144.0,-0.08,1.93,44.42,XY24,NTT,INSURANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,KPIGREEN,731.05,4.41,64.0,H-SC,5.85,124274.0,-1023.0,59950.0,3.07,-0.82,48.24,47.03,141.0,-0.02,0.89,55.60,MH,ATH,POWER
36,INDIAMART,4810.62,-55.57,38.0,H-SC,7.75,123898.0,562.0,131059.0,-0.27,0.46,105.78,106.72,119.0,0.00,0.89,23.94,AR,ATH,MISC
85,WIPRO,420.00,-15.15,43.0,M-LC,6.00,149470.0,-1475.0,110937.0,0.07,-0.98,74.22,72.51,53.0,-0.01,1.07,4.99,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,112.37,64.0,H-SC,12.79,134190.0,-8289.0,27804.0,2.01,-5.82,20.72,13.70,163.0,-0.30,0.96,58.38,OX40N,NTT,BANKS
44,KANSAINER,340.0,-67.06,58.0,H-SC,1.90,226602.0,-43065.0,79401.0,-0.58,-15.97,35.04,13.47,138.0,-0.54,1.62,15.22,XY24,NTT,PAINTS
76,TTKPRESTIG,770.0,94.88,46.0,M-SC,1.72,83925.0,-16852.0,16944.0,-0.24,-16.72,20.19,0.09,245.0,-0.99,0.60,10.26,OX40N,NTT,DURABLES
17,CERA,9475.0,-24.58,37.0,H-SC,2.50,137506.0,-38397.0,80414.0,-0.39,-21.83,58.48,23.89,149.0,-0.48,0.99,18.75,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.21,46.0,H-MC,7.51,102619.0,-29216.0,71443.0,0.59,-22.16,69.62,32.03,98.0,-0.41,0.74,14.94,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,53.0,M-LC,6.03,207708.0,4538.0,37595.0,0.72,2.23,18.10,20.74,55.0,0.12,1.49,7.90,XY25,NTT,FINANCE
31,HINDZINC,730.22,29.80,52.0,M-LC,10.09,211171.0,6095.0,106472.0,-2.96,2.97,50.42,54.89,52.0,0.06,1.51,27.78,X5K,ATH,METALS
78,UNIONBANK,163.00,-11.63,59.0,M-LC,8.79,161033.0,20193.0,24300.0,3.61,14.34,15.09,31.59,66.0,0.83,1.15,44.62,XY24,NTT,BANKS
11,BANKINDIA,190.00,-22.98,65.0,H-MC,12.57,199033.0,19225.0,93187.0,5.12,10.69,46.82,62.52,88.0,0.21,1.43,48.42,XR,NTT,BANKS
84,WHIRLPOOL,2270.00,-40.03,63.0,M-SC,4.16,101332.0,9834.0,68916.0,-2.41,10.75,68.01,86.07,223.0,0.14,0.73,47.15,XR,NTT,DURABLES


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANKINDIA,190.00,-22.98,65.0,H-MC,12.57,199033.0,19225.0,93187.0,5.12,10.69,46.82,62.52,88.0,0.21,1.43,48.42,XR,NTT,BANKS
84,WHIRLPOOL,2270.00,-40.03,63.0,M-SC,4.16,101332.0,9834.0,68916.0,-2.41,10.75,68.01,86.07,223.0,0.14,0.73,47.15,XR,NTT,DURABLES
31,HINDZINC,730.22,29.80,52.0,M-LC,10.09,211171.0,6095.0,106472.0,-2.96,2.97,50.42,54.89,52.0,0.06,1.51,27.78,X5K,ATH,METALS
36,INDIAMART,4810.62,-55.57,38.0,H-SC,7.75,123898.0,562.0,131059.0,-0.27,0.46,105.78,106.72,119.0,0.00,0.89,23.94,AR,ATH,MISC
85,WIPRO,420.00,-15.15,43.0,M-LC,6.00,149470.0,-1475.0,110937.0,0.07,-0.98,74.22,72.51,53.0,-0.01,1.07,4.99,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,14.0,X-LC,9.58,144873.0,-131650.0,241721.0,0.63,-47.61,166.85,39.81,54.0,-0.54,1.04,2.11,XY24,NTT,AUTO
51,QUESS,986.00,-48.21,33.0,X-SC,38.78,51498.0,-13508.0,164438.0,0.45,-20.78,319.31,232.18,198.0,-0.08,0.37,0.45,XY24,NTT,MISC
28,HAVELLS,2069.16,-7.31,40.0,X-MC,2.42,229585.0,-18146.0,93212.0,-1.33,-7.32,40.60,30.30,92.0,-0.19,1.65,4.99,X40,ATH,ELECTRICAL
2,ACC,3906.00,-38.50,42.0,X-MC,3.23,182440.0,-55511.0,208164.0,-0.45,-23.33,114.10,64.15,174.0,-0.27,1.31,2.34,XY24,BTT,CEMENT
9,BAJAJHFL,181.50,-14.69,43.0,X-MC,5.45,179580.0,-20926.0,118074.0,0.06,-10.44,65.75,48.45,90.0,-0.18,1.29,3.56,X40N,ATH,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.04,45.0,H-LC,0.37,155965.0,-30130.0,77608.0,-0.06,-16.19,49.76,25.51,15.0,-0.39,1.12,14.93,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,2.14,61.0,X-MC,1.52,214350.0,22608.0,15155.0,0.17,11.79,7.07,19.69,99.0,1.49,1.54,19.39,XY25,NTT,AC
28,HAVELLS,2069.16,-7.31,40.0,X-MC,2.42,229585.0,-18146.0,93212.0,-1.33,-7.32,40.60,30.30,92.0,-0.19,1.65,4.99,X40,ATH,ELECTRICAL
41,ITC,452.00,-37.50,59.0,X-LC,2.62,201419.0,1281.0,19618.0,-0.06,0.64,9.74,10.44,4.0,0.07,1.44,6.24,X40,NTT,FMCG
33,ICICIGI,2260.25,-13.14,66.0,X-MC,2.68,149391.0,13258.0,17867.0,0.89,9.74,11.96,22.86,91.0,0.74,1.07,23.58,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,33.0,X-SC,38.78,51498.0,-13508.0,164438.0,0.45,-20.78,319.31,232.18,198.0,-0.08,0.37,0.45,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-54.49,14.0,X-LC,9.58,144873.0,-131650.0,241721.0,0.63,-47.61,166.85,39.81,54.0,-0.54,1.04,2.11,XY24,NTT,AUTO
2,ACC,3906.00,-38.50,42.0,X-MC,3.23,182440.0,-55511.0,208164.0,-0.45,-23.33,114.10,64.15,174.0,-0.27,1.31,2.34,XY24,BTT,CEMENT
9,BAJAJHFL,181.50,-14.69,43.0,X-MC,5.45,179580.0,-20926.0,118074.0,0.06,-10.44,65.75,48.45,90.0,-0.18,1.29,3.56,X40N,ATH,FINANCE
73,TCS,4389.96,-29.33,50.0,X-LC,12.75,282094.0,-63862.0,130553.0,1.31,-18.46,46.28,19.28,1.0,-0.49,2.02,4.29,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,33.0,X-SC,38.78,51498.0,-13508.0,164438.0,0.45,-20.78,319.31,232.18,198.0,-0.08,0.37,0.45,XY24,NTT,MISC
55,RELAXO,1176.00,-39.68,43.0,X-SC,4.54,82250.0,-62910.0,138838.0,-0.17,-43.34,168.80,52.31,136.0,-0.45,0.59,10.10,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-35.15,47.0,X-SC,4.27,96861.0,-31809.0,77111.0,1.45,-24.72,79.61,35.20,219.0,-0.41,0.69,11.17,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-6.49,46.0,X-SC,0.85,106434.0,4430.0,39498.0,0.15,4.34,37.11,43.06,122.0,0.11,0.76,22.20,X40N,ATH,MISC
32,HONAUT,58357.33,-22.88,47.0,X-SC,2.56,108750.0,-19188.0,66327.0,-0.17,-15.00,60.99,36.84,143.0,-0.29,0.78,11.33,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-29.33,50.0,X-LC,12.75,282094.0,-63862.0,130553.0,1.31,-18.46,46.28,19.28,1.0,-0.49,2.02,4.29,X40,ATH,IT
39,INFY,2275.00,-19.96,46.0,X-LC,7.62,311811.0,-1077.0,172774.0,1.58,-0.34,55.41,54.87,3.0,-0.01,2.24,6.28,X40,BTT,IT
41,ITC,452.00,-37.50,59.0,X-LC,2.62,201419.0,1281.0,19618.0,-0.06,0.64,9.74,10.44,4.0,0.07,1.44,6.24,X40,NTT,FMCG
82,VBL,671.64,-18.69,52.0,X-LC,10.50,292682.0,-23160.0,135834.0,-0.61,-7.33,46.41,35.67,5.0,-0.17,2.10,5.51,X40N,ATH,FMCG
1,ABB,7934.00,-40.16,51.0,H-LC,6.88,250128.0,-11491.0,130692.0,0.17,-4.39,52.25,45.57,7.0,-0.09,1.79,6.43,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7016.67,66.0,L-SC,6.46,78073.0,-15737.0,92353.0,3.24,-16.78,118.29,81.67,269.0,-0.17,0.56,51.19,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-43.66,52.0,L-SC,29.66,82368.0,-31181.0,71232.0,1.25,-27.46,86.48,35.27,268.0,-0.44,0.59,101.60,XR,NTT,HOTELS
48,MASFIN,398.61,-20.96,43.0,H-SC,9.42,90060.0,-7920.0,29522.0,0.94,-8.08,32.78,22.05,152.0,-0.27,0.65,30.91,XR,ATH,FINANCE
84,WHIRLPOOL,2270.00,-40.03,63.0,M-SC,4.16,101332.0,9834.0,68916.0,-2.41,10.75,68.01,86.07,223.0,0.14,0.73,47.15,XR,NTT,DURABLES
45,KPIGREEN,731.05,4.41,64.0,H-SC,5.85,124274.0,-1023.0,59950.0,3.07,-0.82,48.24,47.03,141.0,-0.02,0.89,55.60,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.0,-187.50,70.0,M-SC,19.63,150777.0,1557.0,142620.0,0.46,1.04,94.59,96.62,208.0,0.01,1.08,63.17,XY25,NTT,FINANCE
4,ANGELONE,3033.0,16.84,71.0,X-SC,8.27,206345.0,15339.0,42363.0,0.80,8.03,20.53,30.21,157.0,0.36,1.48,30.59,X40N,NTT,FINANCE
77,UJJIVANSFB,60.0,112.37,64.0,H-SC,12.79,134190.0,-8289.0,27804.0,2.01,-5.82,20.72,13.70,163.0,-0.30,0.96,58.38,OX40N,NTT,BANKS
67,STARHEALTH,761.0,21.02,66.0,H-SC,9.78,269552.0,-11571.0,144426.0,-1.68,-4.12,53.58,47.26,144.0,-0.08,1.93,44.42,XY24,NTT,INSURANCE
66,SONACOMS,1006.0,-35.24,63.0,M-SC,11.45,80649.0,-20510.0,95400.0,-0.29,-20.27,118.29,74.03,202.0,-0.21,0.58,13.72,AR,BTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.62
1,25,44.24
2,50,75.88


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.26
LC    33.93
MC    21.82
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.55
X40      14.82
XY25     12.42
XR       11.02
X40N     10.27
AR        7.98
OX40N     7.61
X200      1.82
X5K       1.51
SR        1.12
MH        0.89
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.38
X-LC    22.94
X-MC    16.06
M-SC    12.72
M-LC     5.22
H-LC     4.73
X-SC     4.67
H-MC     3.86
M-MC     1.59
L-SC     1.49
L-LC     1.04
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.21,-3.50,37.61
IT,12.71,-18.67,82.73
FINANCE,12.18,-11.36,61.84
MISC,6.87,-19.91,86.35
BANKS,6.50,-9.93,67.76
PAINTS,5.69,-16.17,33.48
ELECTRICAL,5.34,-10.61,49.41
INSURANCE,3.96,-0.02,37.40
AUTO,2.81,-46.65,116.75


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3311516.0,22
XR,1374331.0,14
AR,1217137.0,9
X40,768451.0,10
X40N,595638.0,8
OX40N,585030.0,10
XY25,449789.0,8
SR,266196.0,2
X5K,106472.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3461766.0,25
M-SC,1267527.0,15
X-MC,1088741.0,11
X-LC,1035570.0,13
X-SC,528575.0,6
H-MC,342251.0,3
H-LC,280068.0,3
M-LC,279304.0,4
L-SC,253178.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1188390.0      6
           AR         845852.0      5
           XR         828635.0      7
M-SC       XY24       691809.0      6
X-MC       XY24       588316.0      4
X-LC       X40        473005.0      5
           XY24       306312.0      2
X-SC       X40N       287026.0      4
H-SC       OX40N      272743.0      4
           SR         266196.0      2
X-MC       X40        218335.0      4
H-LC       AR         208300.0      2
X-LC       X40N       190538.0      3
H-MC       XY24       177621.0      1
M-MC       XY24       170330.0      1
X-SC       XY24       164438.0      1
X-MC       XY25       164016.0      2
L-SC       XR         163585.0      2
M-SC       AR         162985.0      2
           OX40N      151251.0      4
           XY25       142620.0      1
           XR         118862.0      2
X-MC       X40N       118074.0      1
M-LC       XR         110937.0      1
           X5K        106472.0      1
H-MC       XR          93187.0      1
L-SC       OX40N       89593.0      1
X-SC       X40         77111.0      1
H-LC       X200        71768.0      1
H-MC       OX40N       71443.0      1
X-LC       XY25        65715.0      3
H-SC       MH          59950.0      1
L-MC       XR          59125.0      1
L-LC       XY25        39843.0      1
M-LC       XY25        37595.0      1
           XY24        24300.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
